In [126]:
from calendar import c
from pathlib import Path
from typing import Protocol, overload, Iterable, Type
import pandas as pd
from dataclasses import dataclass

with open('input.txt') as f:
    lines = f.readlines()

_hand_lines = [(linelist[0], int(linelist[1])) for line in lines if (linelist := line.strip().split())]

df = pd.DataFrame(_hand_lines, columns=["cards", "bid"])

card_order = ["A", "K", "Q", "J", "T", "9", "8", "7", "6", "5", "4", "3", "2"]
card_order_numbered = {card: i for i, card in enumerate(card_order[::-1])}
card_order_joker = card_order_numbered.copy()
card_order_joker["J"]=-1


cards_df = pd.Series(card_order_numbered)

# sorted("A222A", key=lambda x: cards_df[x])


class HandKind(Protocol):
    card_counts: pd.Series 

    def is_kind(self, cards: list[str]) -> bool:
        ...

    def get_rank(self) -> tuple[int, int]:
        ...


class FiveKind(HandKind):
    
    def is_kind(self, cards: list[str]) -> bool:
        #for card in card_order:
        self.card_counts = pd.value_counts(cards)
        if "J" not in cards:
            return len(set(cards)) == 1
        # Now we have a J to test
        return len(set(cards)) <= 2 # either all J or J and one other card type
    
    def get_rank(self) -> tuple[int, int]:
        return cards_df[self.card_counts.idxmax()], 0
    
    
class FourKind(HandKind):

    def is_kind(self, cards: list[str]) -> bool:
        self.card_counts = pd.value_counts(cards)
        if "J" not in cards:
            return self.card_counts.max() == 4
        # Now we have at least one J to test
        return pd.value_counts([card for card in cards if card != "J"]).max() + cards.count("J") == 4

    def get_rank(self) -> tuple[int, int]:
        return cards_df.loc[self.card_counts.idxmax()], 0
    
class FullHouse(HandKind):

    def is_kind(self, cards: list[str]) -> bool:
        self.card_counts = pd.value_counts(cards)
        if not "J" in self.card_counts.index:
            return len(self.card_counts) == 2 and self.card_counts.isin([2, 3]).all()
        # Now we have at least one J to test, full house requires 2, 2, J
        j_less_counts = pd.value_counts([card for card in cards if card != "J"])
        return len(j_less_counts) == 2 and j_less_counts.isin([2, 2]).all()

    def get_rank(self) -> tuple[int, int]:
        return tuple(cards_df[self.card_counts.sort_values(ascending=False).index].to_list())

        #return cards_df.loc[].to_list()
            
class ThreeKind(HandKind):

    def is_kind(self, cards: list[str]) -> bool:
        self.card_counts = pd.value_counts(cards)
        if "J" not in self.card_counts.index:
            return self.card_counts.max() == 3
        # Now we have a J to test
        return pd.value_counts([card for card in cards if card != "J"]).max() + cards.count("J") == 3

    def get_rank(self) -> tuple[int, int]:
        return cards_df.loc[self.card_counts.idxmax()], 0
    
class TwoPair(HandKind):

    def is_kind(self, cards: list[str]) -> bool:
        self.card_counts = pd.value_counts(cards)
        if "J" not in self.card_counts.index:
            return len(self.card_counts) == 3 and self.card_counts.isin([1, 2]).all()
        # Now we have max one J to test, else it would have been 3 of a kind or better
        return pd.value_counts([card for card in cards if card != "J"]).max()==2

    def get_rank(self) -> tuple[int, int]:
        return tuple(sorted(cards_df.loc[self.card_counts[self.card_counts == 2].index].to_list())[::-1])
    
class OnePair(HandKind):

    def is_kind(self, cards: list[str]) -> bool:
        self.card_counts = pd.value_counts(cards)
        if "J" not in self.card_counts.index:
            return self.card_counts.max() == 2
        return pd.value_counts([card for card in cards if card != "J"]).max() + cards.count("J") == 2

    def get_rank(self) -> tuple[int, int]:
        return cards_df.loc[self.card_counts.idxmax()], 0
    
class HighCard(HandKind):

    def is_kind(self, cards: list[str]) -> bool:
        self.card_counts = pd.value_counts(cards)
        return True

    def get_rank(self) -> tuple[int, int]:
        return cards_df.loc[self.card_counts.index.max()], 0
    
    
    
all_hands = [FiveKind, FourKind, FullHouse, ThreeKind, TwoPair, OnePair, HighCard]

all_hands: dict[str, Type[HandKind]] = {hand.__name__: hand for hand in all_hands}

# def five_kind(cards: list[str]) -> bool:
#     if len(set(cards)) == 1:
#         cards_df.loc[cards].max()
#     return 0

# def four_kind(cards: list[str]) -> tuple[int, int]:
#     card_counts = pd.value_counts(cards)
#     if card_counts.max() == 4:
#         return cards_df.loc[card_counts.idxmax()]
#     return 0

# def full_house(cards: list[str]) -> int:
#     card_counts = pd.value_counts(cards)

#     if card_counts.isin([2, 3]).all():


# def three_kind(cards: list[str]) -> bool:
#     return pd.value_counts(cards).max() == 3

# def two_pair(cards: list[str]) -> bool:
#     card_counts = pd.value_counts(cards)
#     return len(card_counts) == 3 and card_counts.isin([1, 2]).all()

# def one_pair(cards: list[str]) -> bool:
#     card_counts = pd.value_counts(cards)
#     return pd.value_counts(cards).max() == 2

# def high_card(cards: list[str]) -> int:
#     return cards_df.loc[cards].max()

# def get_rank(cards: list[str]) -> int:
    
#     if five_kind(cards):
#         return len(cards_df) + 6
#     elif four_kind(cards):
#         return len(cards_df) + 5
#     elif full_house(cards):
#         return len(cards_df) + 4
#     elif three_kind(cards):
#         return len(cards_df) + 3
#     elif two_pair(cards):
#         return len(cards_df) + 2
#     elif one_pair(cards):
#         return len(cards_df) + 1
#     else:
#         return high_card(cards)
    
df["hand"] = df.apply(lambda x: [h for h in x["cards"]], axis=1)

hands = []
hand_kinds = []
ranks = []
for hand in df["hand"]:
    for name, hand_kind in all_hands.items():
        this_hand = hand_kind()
        if this_hand.is_kind(hand):
            hands.append(this_hand)
            hand_kinds.append(name)
            #_h = [h for h in hand]
            #ranks.append(tuple(card_order_numbered[h] for h in hand))
            ranks.append(tuple(card_order_joker[h] for h in hand))
            break

df["hand_kind"] = hands
df["hand_kind_name"] = hand_kinds    
df["simple_rank"] = ranks



# def agg_func_to_calculate_ranks(x):
#     if isinstance(x, list):
#         return x[0]
#     else:
#         return x
    
# df["hand_rank"] = df["hand_kind"].apply(lambda x: x.get_rank())

#df.groupby("hand_kind_name")["hand"].transform(lambda x: x.apply(lambda x: all_hands[x.__class__.__name__].get_rank()))

#df[["hand_kind_name", "hand_rank"]]

# dff: Series[int] = df[["hand_kind_name","hand_rank", "simple_rank"]].groupby("hand_kind_name")["hand_rank"].value_counts()


# df["rank"] = df["hand"].apply(get_rank)

# rank_mapping = {rank: i+1 for i, rank in enumerate(sorted(df["rank"].unique()))}

# df["rank"] = df["rank"].map(rank_mapping)


# #len(df["rank"].value_counts())
# # Should be != 18:

# sum(df["rank"] * df["bid"])


In [110]:
# df[df.cards.str.count("J") > 2] # only 5 or 4 of a kind

# df[df.cards.str.count("J") == 2]
df.head()


,cards,bid,hand,hand_kind,hand_kind_name,simple_rank,hand_rank
0,424KT,464,"[4, 2, 4, K, T]",<__main__.OnePair object at 0x7ff3f4cee790>,OnePair,"(2, 0, 2, 11, 8)","(2, 0)"
1,3J4QA,723,"[3, J, 4, Q, A]",<__main__.TwoPair object at 0x7ff36b45ae50>,TwoPair,"(1, -1, 2, 10, 12)",()
2,94Q85,210,"[9, 4, Q, 8, 5]",<__main__.HighCard object at 0x7ff36774ff10>,HighCard,"(7, 2, 10, 6, 3)","(10, 0)"
3,25722,304,"[2, 5, 7, 2, 2]",<__main__.ThreeKind object at 0x7ff367689a90>,ThreeKind,"(0, 3, 5, 0, 0)","(0, 0)"
4,Q4QQQ,176,"[Q, 4, Q, Q, Q]",<__main__.FourKind object at 0x7ff369591990>,FourKind,"(10, 2, 10, 10, 10)","(10, 0)"


In [125]:
# pd.value_counts([card for card in df["cards"][1] if card != "J"]).max() == 2

In [124]:
# df[["hand_kind_name", "bid", "simple_rank"]].groupby("hand_kind_name").agg({"bid": "sum", "simple_rank": "max"})

In [121]:
df["hand_kind_order"] = df["hand_kind_name"].apply(lambda x: list(all_hands.keys())[::-1].index(x))
sum_bid = 0
for i, bid in enumerate(df.sort_values(by=["hand_kind_order", "simple_rank"])["bid"], start=1):
    sum_bid += i * bid
#    sum_bid += i * bid

print(sum_bid)

253473930


In [123]:

# sum([bid*i for i, bid in enumerate(df.sort_values("rank")["bid"], start=1)])
    

In [122]:
# i = 0
# from collections import defaultdict
# rank_vals = {}
# for hand_type in list(all_hands.keys())[::-1]:

#     hand_type_dict = {}
#     for ind, value in dff.loc[hand_type].sort_index().iteritems():
#         i += 1
#         #print(hand_type, i, ind, value)
#         hand_type_dict[ind] = i
    
#     rank_vals[hand_type] = hand_type_dict

# rank_vals
        
#     #print(hand_type, dff.loc[hand_type])
# for (kind, rank_tuple), count in dff.items():
#     print(kind, rank_tuple, count)